In [1]:
import argparse
import os
import torch
import torch.nn as nn 
import torch.optim as optim
from torchvision import datasets
from torchvision import  models
from torch.autograd import Variable
from tqdm import tqdm 

In [3]:
#data.py

import zipfile
import os
import torchvision.transforms as transforms

# once the images are loaded, how do we pre-process them before being passed into the network


data_transforms = {
    'train':
    transforms.Compose([
        #data augmentation
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        #size of images from pretrained models
        transforms.CenterCrop(size=224),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225]) 
    ]),
    'valid':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
#model.py

import torch
import torch.nn as nn
import torch.nn.functional as F

nclasses = 20 

#choice of the pretrained model
model = models.resnet101(pretrained=True)
#adpating the size of the fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, nclasses)

#We want to finetune on lasts layers and freeze the first layers, that do not contain specific features for our task.
#That's why we first need to know the architecture of the model we chose. 
#As we can see, there are four stages of convolution and one fully connected layer.
#model.eval()

#An idea would be to freeze the three first stages of convolution and finetune on the last stage of convolution and one the fc layer.
#Rq: by default, requires_grad=True, i.e unfrozen (or learnable).
learnable_params = []
for name, module in model.named_children():
    if name not in ['layer3','layer4','fc']:
        for param in module.parameters():
            param.requires_grad = False  #freeze
    if name in ['layer3','layer4','fc']:
        for param in module.parameters():
            learnable_params.append(param)


#Let's check the number of parameters we will train.
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} learnable parameters.')

42,541,140 total parameters.
41,096,212 learnable parameters.


In [6]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [7]:
#check GPU
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')


Using GPU


In [8]:
#choice of some parameters
batch_size = 20
epochs = 20


# Training settings
parser = argparse.ArgumentParser(description='RecVis A3 training script')
parser.add_argument('--data', type=str, default='/content/drive/My Drive/bird_dataset/bird_dataset', metavar='D',
                    help="folder where data is located. train_images/ and val_images/ need to be found in the folder")
parser.add_argument('--batch-size', type=int, default=64, metavar='B',
                    help='input batch size for training (default: 64)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--experiment', type=str, default='experiment', metavar='E',
                    help='folder where experiment outputs are located.')
args, unknown = parser.parse_known_args()
use_cuda = torch.cuda.is_available()
torch.manual_seed(args.seed)

# Create experiment folder
if not os.path.isdir(args.experiment):
    os.makedirs(args.experiment)

# Data initialization and loading
#from data import data_transforms

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args.data + '/train_images',
                         transform=data_transforms['train']),
    batch_size=args.batch_size, shuffle=True, num_workers=1)
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args.data + '/val_images',
                         transform=data_transforms['valid']),
    batch_size=args.batch_size, shuffle=False, num_workers=1)

# Neural network and optimizer
# We define neural net in model.py so that it can be reused by the evaluate.py script
#from model import Net 

if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

optimizer = optim.SGD(learnable_params, lr=0.01, momentum=0.9)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        criterion = torch.nn.CrossEntropyLoss(reduction='mean')
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        # sum up batch loss
        criterion = torch.nn.CrossEntropyLoss(reduction='mean')
        validation_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))


for epoch in range(1, epochs + 1):
    train(epoch)
    validation()
    model_file = args.experiment + '/model_' + str(epoch) + '.pth'
    torch.save(model.state_dict(), model_file)
    print('Saved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file\n')


Using GPU
Train Epoch: 1 [0/1082 (0%)]	Loss: 3.010761
Train Epoch: 1 [640/1082 (59%)]	Loss: 1.987398

Validation set: Average loss: 0.0147, Accuracy: 76/103 (74%)
Saved model to experiment/model_1.pth. You can run `python evaluate.py --model experiment/model_1.pth` to generate the Kaggle formatted csv file

Train Epoch: 2 [0/1082 (0%)]	Loss: 0.915224
Train Epoch: 2 [640/1082 (59%)]	Loss: 0.393403

Validation set: Average loss: 0.0105, Accuracy: 86/103 (83%)
Saved model to experiment/model_2.pth. You can run `python evaluate.py --model experiment/model_2.pth` to generate the Kaggle formatted csv file

Train Epoch: 3 [0/1082 (0%)]	Loss: 0.207317
Train Epoch: 3 [640/1082 (59%)]	Loss: 0.218694

Validation set: Average loss: 0.0086, Accuracy: 89/103 (86%)
Saved model to experiment/model_3.pth. You can run `python evaluate.py --model experiment/model_3.pth` to generate the Kaggle formatted csv file

Train Epoch: 4 [0/1082 (0%)]	Loss: 0.088526
Train Epoch: 4 [640/1082 (59%)]	Loss: 0.026582

V

In [11]:
import argparse
from tqdm import tqdm
import os
import PIL.Image as Image

import torch

#from model import Net

parser = argparse.ArgumentParser(description='RecVis A3 evaluation script')
parser.add_argument('--data', type=str, default='/content/drive/My Drive/bird_dataset/bird_dataset', metavar='D',
                    help="folder where data is located. test_images/ need to be found in the folder")
parser.add_argument('--model', type=str, metavar='M',
                    help="the model file to be evaluated. Usually it is of the form model_X.pth")
parser.add_argument('--outfile', type=str, default='experiment/kaggle.csv', metavar='D',
                    help="name of the output csv file")
parser.add_argument('--experiment', type=str, default='experiment', metavar='E',
                    help='folder where experiment outputs are located.')

args, unknown = parser.parse_known_args()
use_cuda = torch.cuda.is_available()

state_dict = torch.load(args.experiment+'/model_20.pth')
#model_file=args.experiment+'\model_'+str(epoch)+'.pth'
model.load_state_dict(state_dict)
model.eval()
if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

#from data import data_transforms

test_dir = args.data + '/test_images/mistery_category'

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')


output_file = open(args.outfile, "w")
output_file.write("Id,Category\n")

for f in tqdm(os.listdir(test_dir)):
    if 'jpg' in f:
        data = data_transforms['test'](pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
        if use_cuda:
            data = data.cuda()
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1]
        output_file.write("%s,%d\n" % (f[:-4], pred))

output_file.close()

print("Succesfully wrote " + args.outfile + ', you can upload this file to the kaggle competition website')

  1%|          | 5/517 [00:00<00:12, 40.93it/s]

Using GPU


100%|██████████| 517/517 [00:13<00:00, 38.95it/s]

Succesfully wrote experiment/kaggle.csv, you can upload this file to the kaggle competition website
